In [1]:
from skimpy import skim as sk
import re
import os
import json 
import requests 
import pandas as pd
from pandas import json_normalize 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from skimpy import skim


In [2]:
def concat_degradations(degradations):
    files=[file for file in os.listdir(r'D:\Top_gun\Datasets\df_degradations') if file.endswith('.csv')]
    path = r'D:\Top_gun\Datasets\df_degradations'

    all_data = pd.DataFrame()
    for file in files:
        data = pd.read_csv(path + '/' + file)
        all_data = pd.concat([all_data, data])
        # export the data to a csv file
    all_data.to_csv(r'D:\Top_gun\Datasets\df_degradations\all_data_degradations.csv', index=False)
    
concat_degradations('degradations')

In [3]:
df_degradations =  pd.read_csv(r'D:\Top_gun\Datasets\df_degradations\all_data_degradations.csv')
# df_degradations['measure_day'] = df_degradations['measure_day'].astype(str).apply(supprimer_heure_minutes_secondes)
# df_degradations['measure_day'] = pd.to_datetime(df_degradations['measure_day'], format='%Y-%m-%d', errors='coerce')
df_degradations.head()

,ref_deg,linked_aero,compo_concerned,usure_nouvelle,measure_day,need_replacement
0,D003661,E170_6353,REAE170-E170_6353-0,53.308375,2024-06-01,False
1,D009402,E170_6353,REAE170-E170_6353-1,50.111950,2024-06-01,False
2,D000812,E170_6353,SYSE170-E170_6353-2,16.730575,2024-06-01,False
3,D006665,E170_6353,ORDE170-E170_6353-3,6.653075,2024-06-01,False
4,D001163,E170_6353,AUTE170-E170_6353-4,42.033525,2024-06-01,False


In [4]:
df_degradations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1284660 entries, 0 to 1284659
Data columns (total 6 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   ref_deg           1284660 non-null  object 
 1   linked_aero       1284660 non-null  object 
 2   compo_concerned   1284660 non-null  object 
 3   usure_nouvelle    1284660 non-null  float64
 4   measure_day       1284660 non-null  object 
 5   need_replacement  1284660 non-null  bool   
dtypes: bool(1), float64(1), object(4)
memory usage: 50.2+ MB


In [5]:
date = [key for key, value in df_degradations['measure_day'].value_counts().items()]

sorted(date)

['2023-07-02 00:00:00',
 '2023-07-07 00:00:00',
 '2023-08-23 00:00:00',
 '2023-09-04 00:00:00',
 '2024-01-21 00:00:00',
 '2024-04-05 00:00:00',
 '2024-05-31',
 '2024-06-01',
 '2024-06-02',
 '2024-06-03',
 '2024-06-04',
 '2024-06-05',
 '2024-06-06',
 '2024-06-07',
 '2024-06-08',
 '2024-06-09',
 '2024-06-10',
 '2024-06-11',
 '2024-06-12']

In [6]:
skim(df_degradations)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                 Data Types                                                                │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                         │
│ ┃ dataframe         ┃ Values  ┃ ┃ Column Type ┃ Count ┃                                                         │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                         │
│ │ Number of rows    │ 1284660 │ │ string      │ 4     │                                                         │
│ │ Number of columns │ 6       │ │ float64     │ 1     │                                                         │
│ └───────────────────┴─────────┘ │ bool        │ 1     │                                                         │
│                                 └─────────────┴───────┘                                                         │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name        ┃ NA  ┃ NA %   ┃ mean    ┃ sd      ┃ p0    ┃ p25    ┃ p50    ┃ p75    ┃ p100  ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━┩  │
│ │ usure_nouvelle     │   0 │      0 │   32.54 │   17.43 │  0.11 │  17.74 │  32.55 │  47.32 │  91.6 │  ▇▇▇▇▁  │  │
│ └────────────────────┴─────┴────────┴─────────┴─────────┴───────┴────────┴────────┴────────┴───────┴─────────┘  │
│                                                      bool                                                       │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                                ┃ true          ┃ true rate                 ┃ hist              ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩  │
│ │ need_replacement                           │             0 │                         0 │          ▇        │  │
│ └────────────────────────────────────────────┴───────────────┴───────────────────────────┴───────────────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                     ┃ NA     ┃ NA %       ┃ words per row              ┃ total words           ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ ref_deg                         │      0 │          0 │                          1 │               1284660 │  │
│ │ linked_aero                     │      0 │          0 │                          1 │               1284660 │  │
│ │ compo_concerned                 │      0 │          0 │                          1 │               1284660 │  │
│ │ measure_day                     │      0 │          0 │                          1 │               1309275 │  │
│ └─────────────────────────────────┴────────┴────────────┴────────────────────────────┴───────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

In [7]:
# # Enregistrement du fichier degradations dans un fichier csv
# df_degradations.to_csv('D:\\Top_gun\\Data_clean\\degradations_full_cleaned.csv', index=False)